* https://docs.google.com/document/d/1b7DlLtvrVxihcO65dsBaIjUU-a6aLjdQfNHSOM2PhUM/edit?tab=t.0#heading=h.v0phdvfalpr2

## setup and tryout

In [1]:
%load_ext dotenv
%dotenv

```sh
pip install langchain-google-genai
```

In [6]:
import os

# from openai import OpenAI
import jupyter_black

jupyter_black.load()

assert {"OPENAI_API_KEY", "GOOGLE_API_KEY"} <= set(os.environ)

In [2]:
%%time
from langchain_google_genai import ChatGoogleGenerativeAI

# Make sure to set your GOOGLE_API_KEY environment variable
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

response = llm.invoke("What are the best practices for developing with LangChain?")

print(response.content)

Developing with LangChain effectively requires understanding its core concepts, best practices, and common pitfalls. Here's a breakdown of best practices categorized for clarity:

**I. Core Principles and Conceptual Understanding:**

*   **Understand the LangChain Abstractions:**
    *   **Models:**  LLMs (e.g., OpenAI, Cohere), Chat Models, Embeddings. Know their strengths, weaknesses, and cost implications.
    *   **Chains:**  Sequences of calls, often combining models with other components.  Understand different chain types (e.g., Sequential, Router, MapReduce, Stuffing).
    *   **Agents:**  Use LLMs to decide which actions to take. Understand the interplay between agents, tools, and toolkits.
    *   **Memory:**  How to store and manage the history of interactions.  Choose the right memory type (e.g., ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory) based on your needs.
    *   **Indexes:**  Methods for structuring data for LLMs to access.  Und

original:

```python
from langchain_openai.chat_models import ChatOpenAI
# chat = ChatOpenAI(openai_api_key="...")
# If you have an envionrment variable set for OPENAI_API_KEY, you can just do:
chat = ChatOpenAI()
chat.invoke("Hello, how are you?") 
```

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Make sure to set your GOOGLE_API_KEY environment variable.
# You can get one from Google AI Studio: https://aistudio.google.com/app/apikey

# Initialize the Gemini chat model
# You can also specify other models like "gemini-1.5-pro"
chat = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")
# Invoke the model with a prompt
response = chat.invoke("Hello, how are you?")
print(response.content)

## 85. Chat Models -- Coding

In [ ]:
from IPython.display import Markdown

response = chat.invoke("What is the capital of France?")
Markdown(response.content)

In [ ]:
response.response_metadata

original:

```python
from langchain_core.messages import HumanMessage, SystemMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]
result = chat.invoke(messages)
```

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# Ensure your GOOGLE_API_KEY environment variable is set
# 1. Initialize the Gemini chat model
chat = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

# 2. Define the message(s) for the model
text = "What would be a good company name for a company that makes colorful socks?"
messages = [
    SystemMessage(content="You are a helpful assistant that generates company names."),
    HumanMessage(content=text),
]

# 3. Invoke the model with the messages
result = chat.invoke(messages)

# 4. Print the AI's response content
print(result.content)

## 86. Chat Prompt Templates
* https://drive.google.com/file/d/1JoyxZlYfngmXnvrRyo7qqvUoB7qz6il0/view?usp=drive_link

In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate

chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that generates company names"),
        ("human", "{text}"),
    ]
)

result = chat_prompt_template.invoke(
    {
        "text": "What would be a good company name for a company that makes colorful socks?"
    }
)

# model = Cha(model='gpt-4o-mini')

ai_llm_result = chat.invoke(result)
print(ai_llm_result.content)

## 87. Streaming
* https://drive.google.com/file/d/18sGlOZ8AKwON1CXUMnqf9ONfj7bwjSiO/view?usp=drive_link

In [ ]:
import sys
import tqdm, tqdm.notebook

chat = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    # streaming=True,
)
for chunk in tqdm.notebook.tqdm(chat.stream("What is the capital of the moon?")):
    print(chunk.content, end="", flush=True)
    sys.stdout.flush()

## 88. Output Parsers
* https://drive.google.com/file/d/1QWwi3AOCHEoMR83zR21sB7zzKdUxVdfO/view?usp=drive_link

In [ ]:
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [ ]:
class Joke(BaseModel):
    setup: str = Field(description="The setup to the joke")
    punchline: str = Field(description="The punchline to the joke")


class Jokes(BaseModel):
    jokes: List[Joke] = Field(description="A list of jokes")


parser = PydanticOutputParser(pydantic_object=Joke)

In [ ]:
print(parser.get_format_instructions())

In [ ]:
template = "Answer the user query.\n{format_instructions}\n{query}"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])


messages = chat_prompt.invoke(
    {
        "query": "What is a really funny joke about Python programming?",
        "format_instructions": parser.get_format_instructions(),
    }
)

In [ ]:
chat = ChatOpenAI()
## does not work with Gemini
result = chat.invoke(messages)

In [ ]:
try:
    joke_object = parser.parse(result.content)
    print(joke_object.setup)
    print(joke_object.punchline)
except Exception as e:
    print(e)

In [ ]:
chat = ChatOpenAI(model="gpt-4.1-mini")
structured_llm = chat.with_structured_output(Joke)
result = structured_llm.invoke("What is a really funny joke about Python programming?")

In [ ]:
result

In [ ]:
class Joke(BaseModel):
    setup: str = Field(description="The setup to the joke")
    punchline: str = Field(description="The punchline to the joke")
    explanation: str = Field(
        description="A detailed explanation of why this joke is funny."
    )


class Jokes(BaseModel):
    jokes: List[Joke] = Field(description="A list of jokes")

In [ ]:
chat = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
structured_llm = chat.with_structured_output(Joke)
result = structured_llm.invoke("What is a really funny joke about Python programming?")
result

## 89. Summarizing large amounts of text
* https://colab.research.google.com/drive/11t0e03SThhKRPq9T1M7xg6BcooBFaTkA

In [ ]:
GEMINI_MODEL_NAME_CLEVER = "gemini-2.0-flash"
GEMINI_MODEL_NAME_FAST = "gemini-2.0-flash-lite"

### crisp

In [ ]:
# from langchain_openai.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain
from langchain_core.prompts import PromptTemplate

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

llm = ChatGoogleGenerativeAI(
    temperature=0,
    model=GEMINI_MODEL_NAME_CLEVER,
)
chain = load_summarize_chain(llm, chain_type="stuff")

res = chain.invoke(docs)

In [ ]:
res["input_documents"]
Markdown(res["output_text"])

### map reduce

* problem if pages refer to each other (since summaries are done independently)

In [ ]:
from langchain.chains.mapreduce import MapReduceChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import (
    ReduceDocumentsChain,
    MapReduceDocumentsChain,
    StuffDocumentsChain,
)

In [ ]:
llm = ChatGoogleGenerativeAI(temperature=0, model=GEMINI_MODEL_NAME_CLEVER)

# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)

# map_chain:
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# Reduce
reduce_template = """The following is set of summaries:
{doc_summaries}
Take these and distill it into a final, consolidated summary of the main themes.
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [ ]:
# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

In [ ]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

In [ ]:
%%time
# print()
res = map_reduce_chain.invoke(split_docs)

In [ ]:
Markdown(res["output_text"])

### template

In [ ]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain({"input_documents": split_docs}, return_only_outputs=True)

# Page 1 --> Page 2 (Refine) --> Page 3 (Refine)

In [ ]:
for i, v in enumerate(result["intermediate_steps"]):
    display(Markdown(f"## step {i+1}\n{v}"))

In [ ]:
result["output_text"]

## 91. Document Loaders, Text Splitting, Creating LangChain Documents

https://colab.research.google.com/drive/1YdtBCggWStErmFeP5GBSmEaw04kXeKqD